In [2]:
import numpy as np, pandas as pd, scipy.stats as st
from scipy.stats import t, norm
from joblib import Parallel, delayed

In [3]:
data = pd.read_csv('data.csv')

num_obvs = 100_000
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
sector_indices = data['sector'].values
sec_loading = r[sector_indices]
datat = norm.ppf(data['p'])

In [4]:
def process_obs(obs):
    m_factor = obs[0]  
    sec_factor = obs[:len(r)][sector_indices]  
    res_factor = obs[len(r):]  

    control_variate = (
            r[0]**0.5 * m_factor
            + (sec_loading - r[0])**0.5 * sec_factor
            + (1 - sec_loading)**0.5 * res_factor
    )

    ind = control_variate < datat
    loss = np.zeros(len(data))

    if np.any(ind):
        loss[ind] = data.loc[ind, 'm'].values + data.loc[ind, 'd'].values * np.clip(t.rvs(df=3, size=sum(ind)), -5, 5)


    # expected_control = np.mean(control_variate)
    # alpha = np.cov(loss, control_variate)[0,1] / np.var(control_variate)
    # loss -= alpha * (control_variate - expected_control)

    return np.sum(loss), np.var(loss)

In [5]:
factors = np.random.normal(0, 1, (num_obvs, len(r) + len(data)))
antithetic_factors = -factors
combined_factors = np.vstack([factors, antithetic_factors])

answers = Parallel(n_jobs=-1, verbose=5)(delayed(process_obs)(obs) for obs in combined_factors)
answers = np.array(answers)

sample_losses = answers[:, 0]
sample_vars = answers[:, 1]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 1616 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 6928 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 17296 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done 29968 tasks      | elapsed:   14.9s
[Parallel(n_jobs=-1)]: Done 44944 tasks      | elapsed:   20.1s
[Parallel(n_jobs=-1)]: Done 62224 tasks      | elapsed:   26.5s
[Parallel(n_jobs=-1)]: Done 81808 tasks      | elapsed:   34.0s
[Parallel(n_jobs=-1)]: Done 103696 tasks      | elapsed:   41.9s
[Parallel(n_jobs=-1)]: Done 127888 tasks      | elapsed:   57.1s
[Parallel(n_jobs=-1)]: Done 154384 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 183184 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 200000 out of 200000 | elapsed:  1.5min finished


In [8]:
VaR_antithetic = np.percentile(-np.array(sample_losses), 99.9)

In [9]:
print("Średnia wariancji strat:", np.mean(sample_vars))
print("VaR antetyczny (100-ty wynik):", VaR_antithetic)

Średnia wariancji strat: 1341.5054079126141
VaR antetyczny (100-ty wynik): 23639.942384612878


In [ ]:
from scipy.stats import genpareto

data = pd.read_csv('data.csv')

num_obvs = 100_000
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
sector_indices = data['sector'].values
sec_loading = r[sector_indices]
datat = norm.ppf(data['p'])


shift = 2.0  # Tail shift parameter; focus on extreme losses
factors_tail = np.random.normal(shift, 1, (num_obvs, len(r) + len(data)))
antithetic_factors_tail = -factors_tail
combined_factors_tail = np.vstack([factors_tail, antithetic_factors_tail])

answers2 = Parallel(n_jobs=-1, verbose=5)(delayed(process_obs)(obs) for obs in combined_factors_tail)
answers2 = np.array(answers2)

sample_losses2 = answers2[:, 0]
sample_vars2 = answers2[:, 1]


# 1. Define the threshold for extreme losses (e.g., 99th percentile)
threshold = np.percentile(sample_losses, 99)  # Top 1% of losses
print(f"Threshold for GPD: {threshold}")

# 2. Extract excess losses (above the threshold)
excess_losses = sample_losses[sample_losses > threshold] - threshold

# 3. Fit the Generalized Pareto Distribution (GPD) to the excess losses
shape, loc, scale = genpareto.fit(excess_losses)

# 4. Estimate the 99.9% VaR using the fitted GPD
p_tail = 0.999  # Target quantile for VaR
n_excess = len(excess_losses)
VaR_parametric = threshold + scale / shape * ((num_obvs / n_excess * (1 - p_tail)) ** (-shape) - 1)

print(f"Parametric VaR (GPD): {VaR_parametric}")
